<a href="https://colab.research.google.com/github/nadineelnaggar/Plain-RNN-Counter-Experiments/blob/master/Dyck_Generator_Binary_Ternary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import numpy as np
import torch
from collections import defaultdict
import random
from random import randint

In [ ]:
sys.setrecursionlimit(5000)

all_pairs = ['()', '[]', '{}', '<>', '+-', 'ab', 'xo']
all_letters = ''
for elt in all_pairs:
    all_letters += elt

init_ascii = 48  ## corresponds to 0
print(all_letters)

()[]{}<>+-abxo


In [ ]:
class DyckLanguage():
    def __init__(self, num_pairs, p, q):
        self.pair_num = num_pairs
        self.pairs = all_pairs[:num_pairs]
        self.vocabulary = all_letters[:2 * num_pairs]
        self.n_letters = len(self.vocabulary)

        self.openpar = [elt[0] for elt in self.pairs]
        self.closepar = [elt[1] for elt in self.pairs]

        self.p = p
        self.q = q

    # returns the vocabulary
    def return_vocab(self):
        return self.vocabulary

    # generate a sample
    def generate(self, current_size, max_size):
        # Houston, we have a problem here. (Limit exceeded.)
        if current_size >= max_size:
            return ''

        prob = random.random()
        # Grammar: S -> (_i S )_i with prob p | SS with prob q | empty with prob 1 - (p+q)
        if prob < self.p:
            chosen_pair = np.random.choice(self.pairs)  # randomly pick one of the pairs.
            sample = chosen_pair[0] + self.generate(current_size + 2, max_size) + chosen_pair[1]
            if len(sample) <= max_size:
                return sample
        elif prob < self.p + self.q:
            sample = self.generate(current_size, max_size) + self.generate(current_size, max_size)
            if len(sample) <= max_size:
                return sample
        else:
            return ''

        return ''

    # generate 'num' number of samples
    def generate_list(self, num, min_size, max_size):
        arr = []
        size_info = defaultdict(list)
        counter = 0
        while counter < num:
            sample = self.generate(0, max_size)
            if sample not in arr and len(sample) >= min_size:
                counter += 1
                arr.append(sample)
                # print(sample) #extra added by me
                size_info[len(sample)].append(sample)
                if counter % 500 == 0:
                    print('{} samples generated.'.format(counter))

        return arr, size_info

    def output_generator(self, seq):
        output_seq = ''
        stack = []

        for elt in seq:
            dyck_counter = [0 for _ in range(self.pair_num)]

            if elt in self.openpar:
                stack.append(self.closepar[self.openpar.index(elt)])
            else:
                stack.pop()

            if len(stack) > 0:
                index = self.closepar.index(stack[-1])
                dyck_counter[index] = 1

            temp = np.nonzero(dyck_counter)

            binary_code = 0

            for base in temp[0]:
                binary_code += (2 ** (base))

            output_seq += chr(binary_code + init_ascii)

            # print(output_seq) #extra added by me 

        return output_seq

    def depth_counter(self, seq):
        dyck_counter = np.zeros(self.pair_num)
        max_depth = np.zeros((len(seq), self.pair_num))
        counter = 0
        for elt in seq:
            indexl = 0
            if elt in self.openpar:
                indexl = self.openpar.index(elt)
                dyck_counter[indexl] += 1
            else:
                indexl = self.closepar.index(elt)
                dyck_counter[indexl] -= 1
            max_depth[counter] = dyck_counter
            counter += 1
        return max_depth

    def training_set_generator(self, num, min_size, max_size):
        input_arr, input_size_arr = self.generate_list(num, min_size, max_size)
        output_arr = []
        # for seq in input_arr:
        #    output_arr.append(self.output_generator(seq))
        # return input_arr, output_arr, input_size_arr
        ##### CHANGED HERE
        return input_arr, input_size_arr

    # Find letter index from all_letters
    def letterToIndex(self, letter):
        return all_letters.find(letter)

    # Just for demonstration, turn a letter into a <1 x n_letters> Tensor
    def letterToTensor(self, letter):
        tensor = torch.zeros(1, self.n_letters)
        tensor[0][self.letterToIndex(letter)] = 1
        return tensor

    # Turn a line into a <line_length x 1 x n_letters>,
    # or an array of one-hot letter vectors
    def lineToTensor(self, line):
        tensor = torch.zeros(len(line), 1, self.n_letters)
        for li, letter in enumerate(line):
            tensor[li][0][self.letterToIndex(letter)] = 1.0
        return tensor

    def lineToTensorSigmoid(self, line):
        tensor = torch.zeros(len(line), self.n_letters)
        for li, letter in enumerate(line):
            for elt in self.openpar:
                tensor[li][self.letterToIndex(elt)] = 1.0

            binary_code = ord(letter) - init_ascii

            if binary_code > 0:
                for base in range(len(self.closepar) - 1, -1, -1):
                    if binary_code - (2 ** base) >= 0:
                        tensor[li][self.letterToIndex(self.closepar[base])] = 1.0
                        binary_code -= (2 ** base)
        return tensor

In [ ]:
NUM_PAR = 1
MIN_SIZE = 2
MAX_SIZE = 50
P_VAL = 0.5
Q_VAL = 0.25

In [ ]:
Dyck = DyckLanguage (NUM_PAR, P_VAL, Q_VAL)
all_letters = word_set = Dyck.return_vocab ()
n_letters = vocab_size = len (word_set)
print(all_letters)
print(n_letters)

()
2


In [ ]:
# Number of samples in the training corpus
TRAINING_SIZE = 10000
# Number of samples in the test corpus
TEST_SIZE = 5000
print('Loading data...')

training_input, training_input_lengths = Dyck.training_set_generator (TRAINING_SIZE, MIN_SIZE, MAX_SIZE)
print('training input generated')
test_input, test_input_lenghts = Dyck.training_set_generator (TEST_SIZE, MAX_SIZE + 2, 2 * MAX_SIZE)
print('test output generated')





Loading data...
500 samples generated.
1000 samples generated.
1500 samples generated.
2000 samples generated.
2500 samples generated.
3000 samples generated.
3500 samples generated.
4000 samples generated.
4500 samples generated.
5000 samples generated.
5500 samples generated.
6000 samples generated.
6500 samples generated.
7000 samples generated.
7500 samples generated.
8000 samples generated.
8500 samples generated.
9000 samples generated.
9500 samples generated.
10000 samples generated.
training input generated
500 samples generated.
1000 samples generated.
1500 samples generated.
2000 samples generated.
2500 samples generated.
3000 samples generated.
3500 samples generated.
4000 samples generated.
4500 samples generated.
5000 samples generated.
test output generated


In [ ]:
def make_invalid(indices, seqs):
    """
    input a valid sequence and distort it to make it invalid by either
        - swapping the order of opening and closing brackets (equal number but wrong order)
        - replacing opening brackets with closing brackets so that there is an excess of closing brackets
    :return:
    """
    incorrect_order = []
    excess_close = []

    for i in range(len(indices)):

        if i%2 !=0:
            excess_close.append(indices[i])
        elif i%2==0 or i==0:
            incorrect_order.append(indices[i])

    for i in range(len(incorrect_order)):
        seq = seqs[incorrect_order[i]]


        # print('***********************')
        # print('index in original array = ', incorrect_order[i])
        # print(seq)
        # print(len(seq))
        # print(seq[0])
        # print(seq.find('('))
        seq = seq[-1:]+seq[:-1]
        # print(seq)
        seqs[incorrect_order[i]]=seq

    for i in range(len(excess_close)):
        # print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        # print('index in original array = ', excess_close[i])
        seq = seqs[excess_close[i]]
        # print(seq)
        # print(len(seq))
        # print(seq[0])
        ind = seq.find('()')
        # print(ind)
        # # seq[ind]= ')'
        # # seq[ind+1]='('
        # print(ind+1)
        seq = seq[:ind]+')'+seq[ind+1:]
        # print(seq)
        seqs[excess_close[i]] = seq
    return seqs


In [ ]:
def make_incomplete(indices, seqs):
    """
    input a valid sequence and distort it to make it potentially valid by
        - replacing one or more closing brackets in a random location with an opening bracket
    :return:
    """

    for i in range(len(indices)):
        seq = seqs[indices[i]]


        # print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
        # print('index in original array = ', indices[i])
        # print(seq)
        # print(len(seq))
        # print(seq[0])
        # # print(seq.find(')'))

        ind = seq.find(')')
        # print(ind)
        # # seq[ind]= ')'
        # # seq[ind+1]='('
        # print(ind + 1)
        seq = seq[:ind] + '(' + seq[ind + 1:]
        # print(seq)
        # # seqs[excess_close[i]] = seq
        #
        # # seq = seq[-1:]+seq[:-1]
        # # print(seq)
        seqs[indices[i]]=seq
    return seqs

In [ ]:
def generateBinaryDataset(seqs):
    
    invalid_indices = []
    incomplete_indices = []
    labels = []
    labels_timestep = []
    for i in range(len(seqs)):
        labels.append('valid')

    # make 1/3 of invalid sequences have excess opening, 1/3 have excess closing, and 1/3 have incorrect order
    for i in range(100000):
        value = randint(1, len(seqs) - 1)
        if value not in invalid_indices and value not in incomplete_indices and len(invalid_indices) < len(seqs) / 3:
            invalid_indices.append(value)
        value2 = randint(1, len(seqs) - 1)
        if value2 not in invalid_indices and value2 not in incomplete_indices and len(incomplete_indices) < len(
                seqs) / 6:
            incomplete_indices.append(value2)

        if len(incomplete_indices) == len(invalid_indices) == len(seqs) / 3:
            break

    # print(invalid_indices)
    # print(incomplete_indices)
    # print(len(invalid_indices))
    # print(len(incomplete_indices))

    make_invalid(invalid_indices, seqs)
    make_incomplete(incomplete_indices, seqs)

    for i in range(len(incomplete_indices)):
        labels[incomplete_indices[i]] = 'invalid'

    for i in range(len(invalid_indices)):
        labels[invalid_indices[i]] = 'invalid'
    

    for i in range(len(seqs)):
      seq = seqs[i]
      count = 0
      output_seq = ''

      for index, char in enumerate(seq):

        if char == '(':
          # print(char)
          if count>0:
            count+=1
            output_seq = output_seq+'1'
            # print(output_seq)
          elif count==0:
            count+=1
            output_seq = output_seq+'1'
            # print(output_seq)
          elif count==-1:
            output_seq = output_seq+'1'
            # print(output_seq)
        elif char==')':
          # print(char)
          if count>0:
            count-=1
            if count>0:
              output_seq = output_seq+'1'
            elif count==0:
              output_seq = output_seq+'0'
            # print(output_seq)
          # elif count>1:
          #   count-=1
          #   output_seq = output_seq+'0'
          # elif count==0:
          #   count=-1
          #   output_seq = output_seq+'0'
          #   print(output_seq)
          elif count<=0:
            output_seq = output_seq+'1'
            # print(output_seq)
      labels_timestep.append(output_seq)
      # for index, char in enumerate(seq):

      #   if char == '(':
      #     if count>0:
      #       count+=1
      #       output_seq = output_seq+'1'
      #     elif count==0:
      #       count+=1
      #       output_seq = output_seq+'1'
      #     elif count==-1:
      #       output_seq = output_seq+'1'
      #   elif char==')':
      #     if count>0:
      #       count-=1
      #       output_seq = output_seq+'1'
      #     # elif count>1:
      #     #   count-=1
      #     #   output_seq = output_seq+'0'
      #     elif count==0:
      #       count=-1
      #       output_seq = output_seq+'0'
      #     elif count==-1:
      #       output_seq = output_seq+'1'
        # labels_timestep.append(output_seq)

        # if count>0:
        #   output_seq = output_seq+'1'
        #   if char=='(':
        #     count+=1
        #   elif char==')':
        #     count-=1
        # elif count==0:
        #   output_seq=output_seq+'0'
        #   if char=='(':
        #     count+=1
        #   elif char==')':
        #     count = -1
        # elif count==-1:
        #   output_seq=output_seq+'1'




    # document_name = 'Dyck1_BinaryDataset_'+str(start_n)+'to'+str(end_n)+'pairs_'+str(dataset_size)+'elements_balanced.txt'
    # with open(document_name,'a') as f:
    #     for i in range(len(seqs)):
    #         f.write(seqs[i] + ',' + labels[i] + '\n')
    # print('',document_name,' completed')
    return seqs, labels, labels_timestep

In [ ]:
def generateTernaryDataset(seqs):

    invalid_indices = []
    incomplete_indices = []
    labels = []
    labels_timestep = []
    for i in range(len(seqs)):
        labels.append('valid')

    for i in range(100000):
        value = randint(1, len(seqs) - 1)
        if value not in invalid_indices and value not in incomplete_indices and len(invalid_indices) < len(seqs) / 3:
            invalid_indices.append(value)
        value2 = randint(1, len(seqs) - 1)
        if value2 not in invalid_indices and value2 not in incomplete_indices and len(incomplete_indices) < len(
                seqs) / 3:
            incomplete_indices.append(value2)

        if len(incomplete_indices) == len(invalid_indices) == len(seqs) / 3:
            break

    # print(invalid_indices)
    # print(incomplete_indices)
    # print(len(invalid_indices))
    # print(len(incomplete_indices))

    make_invalid(invalid_indices, seqs)
    make_incomplete(incomplete_indices, seqs)

    for i in range(len(incomplete_indices)):
        labels[incomplete_indices[i]] = 'incomplete'

    for i in range(len(invalid_indices)):
        labels[invalid_indices[i]] = 'invalid'
    # document_name = 'Dyck1_TernaryDataset_'+str(start_n)+'to'+str(end_n)+'pairs_'+str(dataset_size)+'elements_balanced.txt'
    # with open(document_name,'a') as f:
    #     for i in range(len(seqs)):
    #         f.write(seqs[i] + ',' + labels[i] + '\n')
    # print('',document_name,' completed')

    for i in range(len(seqs)):
      seq = seqs[i]
      count = 0
      output_seq = ''
      for index, char in enumerate(seq):
        
      # for index, char in enumerate(seq):

        if char == '(':
          # print('char = ',char)
          # print('count = ',count)
          if count>0:
            count+=1
            output_seq = output_seq+'1'
            # print(output_seq)
          elif count==0:
            count+=1
            output_seq = output_seq+'1'
            # print(output_seq)
          elif count==-1:
            output_seq = output_seq+'2'
            # print(output_seq)
        elif char==')':
          # print('char = ',char)
          # print('count = ',count)
          if count>0:
            count-=1
            if count>0:
              output_seq = output_seq+'1'
            elif count==0:
              output_seq = output_seq+'0'
            # print(output_seq)
          elif count==0:
            count=-1
            output_seq = output_seq+'2'
            # print(output_seq)
          # elif count>1:
          #   count-=1
          #   output_seq = output_seq+'0'
          # elif count==0:
          #   count=-1
          #   output_seq = output_seq+'0'
          #   print(output_seq)
          elif count<0:
            output_seq = output_seq+'2'
      labels_timestep.append(output_seq)

        # if char == '(':
        #   if count>0:
        #     count+=1
        #     output_seq = output_seq+'1'
        #   elif count==0:
        #     count+=1
        #     output_seq = output_seq+'0'
        #   elif count==-1:
        #     output_seq = output_seq+'1'
        # elif char==')':
        #   if count>1:
        #     count-=1
        #     output_seq = output_seq+'1'
        #   elif count>1:
        #     count-=1
        #     output_seq = output_seq+'0'
        #   elif count==0:
        #     count=-1
        #     output_seq = output_seq+'1'
        #   elif count==-1:
        #     output_seq = output_seq+'1'
        # labels_timestep.append(output_seq)

  
    return seqs, labels, labels_timestep

In [ ]:
training_input_binary, training_output_binary, training_output_binary_timestep = generateBinaryDataset(training_input)
print('training input labels generated')
test_input_binary, test_output_binary, test_output_binary_timestep = generateBinaryDataset(test_input)
print('test input labels generated')

training input labels generated
test input labels generated


In [ ]:
print('Binary training data generated, writing training set to document')
with open('Dyck1_Dataset_Binary_train_.txt', 'a') as f:
  for i in range(len(training_input_binary)):
    # f.write(str(training_input[i])+','+str(training_output[i])+','+str(training_input_lengths[i])+'\n')
    f.write(str(training_input_binary[i])+','+str(training_output_binary[i])+','+training_output_binary_timestep[i]+'\n')



# training_input, training_output, training_input_lengths = Dyck.training_set_generator (TRAINING_SIZE, MIN_SIZE, MAX_SIZE)
# print('training data generated, writing training set to document')
# with open('Dyck1_Dataset_Suzgun_train.txt', 'a') as f:
#   for i in range(len(training_input)):
#     f.write(str(training_input[i])+','+str(training_output[i])+','+str(training_input_lengths[i])+'\n')

print('binary train set written to document')


print('binary test data generated, writing test set to document')
# test_input, test_output, test_input_lengths = Dyck.training_set_generator (TEST_SIZE, MAX_SIZE + 2, 2 * MAX_SIZE)
with open('Dyck1_Dataset_Binary_test_.txt', 'a') as f:
  for i in range(len(test_input)):
    # f.write(str(test_input[i])+','+str(test_output[i])+','+str(test_input_lengths[i])+'\n')
    f.write(str(test_input_binary[i])+','+str(test_output_binary[i])+','+test_output_binary_timestep[i]+'\n')

print('test set written to document')

Binary training data generated, writing training set to document
binary train set written to document
binary test data generated, writing test set to document
test set written to document


In [ ]:
training_input_ternary, training_output_ternary, training_output_ternary_timestep = generateTernaryDataset(training_input)
print('training input labels generated')
test_input_ternary, test_output_ternary, test_output_ternary_timestep = generateTernaryDataset(test_input)
print('test input labels generated')

training input labels generated
test input labels generated


In [ ]:
print('Ternary training data generated, writing training set to document')
with open('Dyck1_Dataset_Ternary_train_.txt', 'a') as f:
  for i in range(len(training_input_ternary)):
    # f.write(str(training_input[i])+','+str(training_output[i])+','+str(training_input_lengths[i])+'\n')
    f.write(str(training_input_ternary[i])+','+str(training_output_ternary[i])+','+training_output_ternary_timestep[i]+'\n')



# training_input, training_output, training_input_lengths = Dyck.training_set_generator (TRAINING_SIZE, MIN_SIZE, MAX_SIZE)
# print('training data generated, writing training set to document')
# with open('Dyck1_Dataset_Suzgun_train.txt', 'a') as f:
#   for i in range(len(training_input)):
#     f.write(str(training_input[i])+','+str(training_output[i])+','+str(training_input_lengths[i])+'\n')

print('ternary train set written to document')


print('ternary test data generated, writing test set to document')
# test_input, test_output, test_input_lengths = Dyck.training_set_generator (TEST_SIZE, MAX_SIZE + 2, 2 * MAX_SIZE)
with open('Dyck1_Dataset_Ternary_test_.txt', 'a') as f:
  for i in range(len(test_input)):
    # f.write(str(test_input[i])+','+str(test_output[i])+','+str(test_input_lengths[i])+'\n')
    f.write(str(test_input_ternary[i])+','+str(test_output_ternary[i])+','+test_output_ternary_timestep[i]+'\n')

print('test set written to document')

Ternary training data generated, writing training set to document
ternary train set written to document
ternary test data generated, writing test set to document
test set written to document


In [ ]:
print('done')

done


In [ ]:
      seq = '((()))'
      seq = '()()()'
      seq = '(())()'
      # seq = '))('
      seq = '((()))(('
      
      count = 0
      output_seq = ''
      for index, char in enumerate(seq):

        if char == '(':
          print(char)
          if count>0:
            count+=1
            output_seq = output_seq+'1'
            print(output_seq)
          elif count==0:
            count+=1
            output_seq = output_seq+'1'
            print(output_seq)
          elif count==-1:
            output_seq = output_seq+'1'
            print(output_seq)
        elif char==')':
          print(char)
          if count>0:
            count-=1
            if count>0:
              output_seq = output_seq+'1'
            elif count==0:
              output_seq = output_seq+'0'
            print(output_seq)
          # elif count>1:
          #   count-=1
          #   output_seq = output_seq+'0'
          # elif count==0:
          #   count=-1
          #   output_seq = output_seq+'0'
          #   print(output_seq)
          elif count<=0:
            output_seq = output_seq+'1'
            print(output_seq)
      print(output_seq)

(
1
(
11
(
111
)
1111
)
11111
)
111110
(
1111101
(
11111011
11111011


In [ ]:
      seq = '((()))'
      # seq = '()()()'
      # seq = '(())()'
      # seq = '))('
      seq = '((()))(('
          
      count = 0
      output_seq = ''
      for index, char in enumerate(seq):

        if char == '(':
          print('char = ',char)
          print('count = ',count)
          if count>0:
            count+=1
            output_seq = output_seq+'1'
            print(output_seq)
          elif count==0:
            count+=1
            output_seq = output_seq+'1'
            print(output_seq)
          elif count==-1:
            output_seq = output_seq+'2'
            print(output_seq)
        elif char==')':
          print('char = ',char)
          print('count = ',count)
          if count>0:
            count-=1
            if count>0:
              output_seq = output_seq+'1'
            elif count==0:
              output_seq = output_seq+'0'
            print(output_seq)
          elif count==0:
            count=-1
            output_seq = output_seq+'2'
            print(output_seq)
          # elif count>1:
          #   count-=1
          #   output_seq = output_seq+'0'
          # elif count==0:
          #   count=-1
          #   output_seq = output_seq+'0'
          #   print(output_seq)
          elif count<0:
            output_seq = output_seq+'2'
            print(output_seq)
          print('final count = ',count)
          print('final output_seq = ',output_seq)
      print(output_seq)

char =  (
count =  0
1
char =  (
count =  1
11
char =  (
count =  2
111
char =  )
count =  3
1111
final count =  2
final output_seq =  1111
char =  )
count =  2
11111
final count =  1
final output_seq =  11111
char =  )
count =  1
111110
final count =  0
final output_seq =  111110
char =  (
count =  0
1111101
char =  (
count =  1
11111011
11111011


In [ ]:
print('test')

test
